In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Load the datasets
train_df = pd.read_csv('mnist_train.csv')
test_df = pd.read_csv('mnist_test.csv')

# Separate features and labels
X_train = train_df.drop('label', axis=1).values
y_train = train_df['label'].values
X_test = test_df.drop('label', axis=1).values
y_test = test_df['label'].values

# Normalize the pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Reshape the data to fit the model
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((48000, 28, 28, 1),
 (48000, 10),
 (12000, 28, 28, 1),
 (12000, 10),
 (10000, 28, 28, 1),
 (10000, 10))

In [ ]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)

test_loss, test_acc

Epoch 1/10
1500/1500 [==============================] - 52s 34ms/step - loss: 0.3034 - accuracy: 0.9078 - val_loss: 0.0638 - val_accuracy: 0.9809
Epoch 2/10
1500/1500 [==============================] - 50s 34ms/step - loss: 0.0974 - accuracy: 0.9727 - val_loss: 0.0450 - val_accuracy: 0.9868
Epoch 3/10
1500/1500 [==============================] - 47s 32ms/step - loss: 0.0718 - accuracy: 0.9791 - val_loss: 0.0400 - val_accuracy: 0.9887
Epoch 4/10
1500/1500 [==============================] - 50s 33ms/step - loss: 0.0550 - accuracy: 0.9845 - val_loss: 0.0475 - val_accuracy: 0.9868
Epoch 5/10
1500/1500 [==============================] - 47s 31ms/step - loss: 0.0457 - accuracy: 0.9877 - val_loss: 0.0432 - val_accuracy: 0.9875
Epoch 6/10
1500/1500 [==============================] - 48s 32ms/step - loss: 0.0387 - accuracy: 0.9893 - val_loss: 0.0403 - val_accuracy: 0.9900
Epoch 7/10
1500/1500 [==============================] - 48s 32ms/step - loss: 0.0320 - accuracy: 0.9910 - val_loss: 0.0388 -

(0.03586764261126518, 0.9926999807357788)